In [1]:
# 20做了分片 5个 这个是不分片的

In [2]:
import tensorflow as tf
import os
import random
import sys
import math
from PIL import Image
import numpy as np

In [4]:
# 验证集数量
_NUM_TEST = 500
# 随机种子
_RANDOM_SEED = 0

# 数据块路径 其实就是存放图片的路径
DATASET_DIR = 'G:/Anaconda3/My_Jupyter/captcha/images/'
# 标签文件名称
TFRECORD_DIR = 'G:/Anaconda3/My_Jupyter/captcha/'


# 判断 tfrecord 文件是否存在
def _dataset_exists(dataset_dir):
    for split_name in ['train','test']:
        output_filename = os.path.join(dataset_dir,split_name + '.tfrecords')
        if not tf.gfile.Exists(output_filename):
            return False
    return True
    

# 获取所有的验证码图片
def _get_filenames_and_classes(dataset_dir):            
    photo_filenames = []
    for filename in os.listdir(dataset_dir):
        path = os.path.join(dataset_dir,filename)
        photo_filenames.append(path)
    return photo_filenames
        
    
    
# 文本转化为整数格式
def int64_feature(values):
    if not isinstance(values,(tuple,list)):
        values = [values]
    return tf.train.Feature(int64_list = tf.train.Int64List(value = values))
    
# 文本转化为字节格式
def bytes_featurn(values):
    return tf.train.Feature(bytes_list = tf.train.BytesList(value = [values]))


# 转化tfexample 固定写法
def image_to_tfexample(image_data, label0, label1, label2, label3):
    # abstract base class for protocol messages
    return tf.train.Example(features = tf.train.Features(feature = {
        'image':bytes_featurn(image_data),
        'label0':int64_feature(label0),
        'label1':int64_feature(label1),
        'label2':int64_feature(label2),
        'label3':int64_feature(label3),
    }))

# def write_label_file(labels_to_class_names,dataset_dir,filename = LABELS_FILENAME):
#     labels_filename = os.path.join(dataset_dir,filename)
#     with tf.gfile.Open(labels_filename,'w') as f:
#         for label in labels_to_class_names:
#             class_name = labels_to_class_names[label]
#             f.write('%d:%s\n'%(label,class_name))  



#把数据转为TFRecord格式  

# #数据转换  （你是训练数据还是测试数据、训练集或是测试集、分类字典、图片路径）
# _convert_dataset('train',training_filenames,DATASET_DIR)
# _convert_dataset('test',testing_filenames,DATASET_DIR)

# 标签文件名称
# TFRECORD_DIR = 'G:/Anaconda3/My_Jupyter/captcha/'

def _convert_dataset(split_name,filenames,dataset_dir):
    
    # assert 断言 assert expression 相当于 if not expression raise AssertionError
    assert split_name in ['train','test']   # 先判断一下split_name是不是训练集或者测试集

    with tf.Session() as sess:
        
        # 定义一个tfrecord文件的路径+名字
        output_filename = os.path.join(TFRECORD_DIR,split_name + '.tfrecords')

        # 这句是一个固定套路 把我们定义好的tfrecord文件的路径+名字 传入
        with tf.python_io.TFRecordWriter(output_filename) as tfrecore_writer:
            for i,filename in enumerate(filenames):
                
                # filenames是testing_filenames = photo_filenames[:_NUM_TEST]  # 分配给测试集或者训练集的经过打乱的所有图片的路径
                
                try:   # 防止有些图片可能损坏 打印出来跳过就行
                    sys.stdout.write('\r>>%s  Converting image %d/%d' % (split_name, i+1,len(filenames)))  
                    sys.stdout.flush()   # 这里打印一些信息 处理到了当前哪个图片 数据块是哪个数据块

                    # 读取图片  路径
                    image_data = Image.open(filename)     # 用python中处理图片的包打开图片
                    # 根据模型的结构resize
                    image_data = image_data.resize((224,224))    # 网络要求224×224格式读入 imresize
                    # 灰度化
                    image_data = np.array(image_data.convert('L'))    # 图片的颜色对我们的识别是没有帮助的 灰度化
                    # 将图片转化为bytes
                    image_data = image_data.tobytes()           # 等会要调用image_to_tfexample函数 把图片存成某种（byte、int、float）格式才能保存


                    # 获取label    根据文件名和/ 进行切分 拿到它最后的部分 再拿到它的第0到第4位
                    labels = filename.split('/')[-1][0:4]
                    num_labels = []
                    for j in range(4):
                        num_labels.append(int(labels[j]))

                    #生成tfrecord文件       # 图片信息 格式 第几类    # 多任务学习的方式 把这四个数分开
                    example = image_to_tfexample(image_data, num_labels[0],num_labels[1],num_labels[2],num_labels[3])  
                    tfrecore_writer.write(example.SerializeToString())   # 写入对应位置

                except IOError as e:
                    print("Could not read: ",filename)
                    print("Error: ",e)
                    print("Skip it \n")
    sys.stdout.write('\n')  
    sys.stdout.flush()  
    
    
    
if __name__=='__main__':  
    # 判断tfrecord文件是否存在  如果存在 就直接跳过了 不用管它了
    if _dataset_exists(TFRECORD_DIR):
        print('tfrecord 文件已经存在')

    else :  
        # 从DATASET_DIR = 'G:/Anaconda3/My_Jupyter/captcha/images/'路径下 获取所有图片的路径
        photo_filenames = _get_filenames_and_classes(DATASET_DIR)

        # 把数据切分为训练集和测试集，并打乱
        random.seed(_RANDOM_SEED)     # 给一个种子 看不懂 这个基本上没用吧 为了让生成的随机数在一个盆子里 但是这里只有一个盆子
        random.shuffle(photo_filenames)    # 调用shuffle 可以把list打乱
        testing_filenames = photo_filenames[:_NUM_TEST]  # 分配给测试集
        training_filenames = photo_filenames[_NUM_TEST:]  # 分配给训练集

        #数据转换  （你是训练数据还是测试数据、训练集或是测试集、分类字典、图片路径）
        _convert_dataset('train',training_filenames,DATASET_DIR)  
        _convert_dataset('test',testing_filenames,DATASET_DIR)  

        print('生成tfrecord文件')

>>train  Converting image 5787/5787
>>test  Converting image 500/500
生成tfrecord文件


In [ ]:
# 看形状 sess.run(tf.shape(net0))